In [2]:
# =============================================================================
# =============================================================================
# # 문제 04 유형(DataSet_04.csv 이용)
#
#구분자 : comma(“,”), 6,718 Rows, 4 Columns, UTF-8 인코딩

# 한국인의 식생활 변화가 건강에 미치는 영향을 분석하기에 앞서 육류
# 소비량에 대한 분석을 하려고 한다. 확보한 데이터는 세계 각국의 1인당
# 육류 소비량 데이터로 아래와 같은 내용을 담고 있다.

# 컬 럼 / 정 의 / Type
# LOCATION / 국가명 / String
# SUBJECT / 육류 종류 (BEEF / PIG / POULTRY / SHEEP) / String
# TIME / 연도 (1990 ~ 2026) / Integer
# Value / 1인당 육류 소비량 (KG) / Double
# =============================================================================
# =============================================================================

# (참고)
# #1
# import pandas as pd
# import numpy as np
# #2
# from scipy.stats import ttest_rel
# #3
# from sklearn.linear_model import LinearRegression

import pandas as pd
import numpy as np

data4 = pd.read_csv('./DataSet/DataSet_04.csv')
data4.columns
# ['LOCATION', 'SUBJECT', 'TIME', 'Value']

Index(['LOCATION', 'SUBJECT', 'TIME', 'Value'], dtype='object')

In [13]:
# =============================================================================
# 1.한국인의 1인당 육류 소비량이 해가 갈수록 증가하는 것으로 보여 상관분석을 통하여
# 확인하려고 한다. 
# - 데이터 파일로부터 한국 데이터만 추출한다. 한국은 KOR로 표기되어 있다.
# - 년도별 육류 소비량 합계를 구하여 TIME과 Value간의 상관분석을 수행하고
# 상관계수를 소수점 셋째 자리에서 반올림하여 소수점 둘째 자리까지만 기술하시오. 
# (답안 예시) 0.55
# =============================================================================

# 1. 한국 데이터만 추출
q1 = data4[data4.LOCATION == 'KOR']

# 년도별 육류 소비량 합계
q1_df = q1.groupby('TIME')['Value'].sum().reset_index()

# TIME과 Value간의 상관분석
result = q1_df.corr()['TIME']['Value']
print(f'답 : {round(result, 2)}')

# 답 : 0.96

답 : 0.96


In [23]:
# =============================================================================
# 2. 한국 인근 국가 가운데 식생의 유사성이 상대적으로 높은 일본(JPN)과 비교하여, 연도별
# 소비량에 평균 차이가 있는지 분석하고자 한다.
# - 두 국가의 육류별 소비량을 연도기준으로 비교하는 대응표본 t 검정을 수행하시오.
# - 두 국가 간의 연도별 소비량 차이가 없는 것으로 판단할 수 있는 육류 종류를 모두
# 적으시오. (알파벳 순서) (답안 예시) BEEF, PIG, POULTRY, SHEEP
# =============================================================================

# 1. 한국, 일본 데이터 추출
q4_2 = data4[data4['LOCATION'].isin(['KOR','JPN'])]

# 2. 육류목록 추출
sub_list = q4_2.SUBJECT.unique()

# 3. 반복문 이용해서 육류 종류별 대응 t 검정
from scipy.stats import ttest_rel

q4_2out = []
for i in sub_list:
    # 육류별
    temp = q4_2[q4_2['SUBJECT'] == i]
    # pivot table 만들기
    temp2 = pd.pivot_table(temp, index='TIME', columns='LOCATION',
                  values = 'Value', aggfunc='mean').dropna()
    # ttest 수행
    ttest_out = ttest_rel(temp2['KOR'],temp2['JPN'])
    # 육류별 pvalue 리스트
    q4_2out.append([i,ttest_out.pvalue])

q4_2_out2 = pd.DataFrame(q4_2out, columns=['sub','pvalue'])

# 4. 두 국가 간의 연도별 소비량 차이가 없는 것으로 판단할 수 있는 육류 종류
# H0 : 소비량 차이가 있다
# H1 : 소비량 차이가 없다
q4_2_out2[q4_2_out2.pvalue >= 0.05]['sub']

# 답 : POULTRY

2    POULTRY
Name: sub, dtype: object

1. `ttest_ind()` : 독립표본 t 검정
2. `ttest_rel()` : 대응표본 t 검정
3. `ttest_1samp()` : 단일표본 t 검정

In [30]:
# =============================================================================
# 3.(한국만 포함한 데이터에서) Time을 독립변수로, Value를 종속변수로 하여 육류
# 종류(SUBJECT) 별로 회귀분석을 수행하였을 때, 가장 높은 결정계수를 가진 모델의
# 학습오차 중 MAPE를 반올림하여 소수점 둘째 자리까지 기술하시오. (답안 예시) 21.12
# (MAPE : Mean Absolute Percentage Error, 평균 절대 백분율 오차)
# (MAPE = Σ ( | y - y ̂ | / y ) * 100/n ))
#
# =============================================================================

# 1. 한국만 포함한 데이터
q4_3 = data4[data4['LOCATION'] == 'KOR']

# 2. 육류 종류별
sub_list = q4_3.SUBJECT.unique()

# 3. 육류 종류별 회귀분석
from sklearn.linear_model import LinearRegression

q4_3_out = []
for i in sub_list:
    # 육류별
    temp = q4_3[q4_3.SUBJECT == i]
    # 회귀분석 학습
    lm = LinearRegression().fit(temp[['TIME']], temp['Value'])
    # 회귀분석 결과
    r2 = lm.score(temp[['TIME']], temp['Value'])
    # 예측
    pred = lm.predict(temp[['TIME']])
    # (MAPE = Σ ( | y - y ̂ | / y ) * 100/n )
    mape = (abs(temp['Value'] - pred) / temp['Value']).sum() * 100 / len(temp)
    q4_3_out.append([i, r2, mape])
    
# 4. 가장 높은 결정계수를 가진 모델의 학습오차 중 MAPE를 반올림하여 소수점 둘째 자리까지
q4_3_df = pd.DataFrame(q4_3_out, columns=['sub','r2','mape'])

q4_3_df.loc[q4_3_df.r2.idxmax(), 'mape']

# 답 : 5.78

5.783357902874552

lm = linearRegression()

lm.score() : 결정 계수 $R^2$ 